# F&B 20대 추천점수 산정 


1. F&B 태그/데이터랩 데이터 정제 및 상위 키워드 확인
2. F&B 데이터의 네이버 데이터랩 20대 점수 기준으로 필터링 
3. F&B 데이터 EDA 20대점수 상위권, 하위권 나누어서 비교 
4. F&B 데이터 추천 점수 산정

In [1]:
import pandas as pd
import numpy as np
import re
import requests
import json
import matplotlib.pyplot as plt
import collections
import ast
import itertools
import plotly.express as px

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
fb_data = pd.read_csv('data/F&B데이터_co2_final.csv')

## 1. F&B 태그/데이터랩 데이터 탐색



In [4]:
# 키워드 전처리
def keyword_processing(tags):
    tags = tags.replace('#', '').strip()
    tags = tags.replace(',', '').strip()
    tags = tags.replace('/', '').strip()
    tags = tags.replace("  ", ' ').strip()
    return tags

In [5]:
fb_data['tags'] = list(map(lambda x : keyword_processing(x), fb_data['tags']))

### (1) F&B 비짓제주 태그 데이터 

In [7]:
from collections import Counter

In [8]:
tag_list = []
for i in range(len(fb_data)):
    tags = fb_data['tags'][i].split()
    for tag in tags:
        tag_list.append(tag)

In [9]:
# 태그 데이터 개수 확인
word_count = Counter(tag_list)
len(word_count)

1822

In [10]:
# 상위 태그 180개 확인
word_count.most_common(30)

[('음식', 1599),
 ('식당', 413),
 ('카페', 328),
 ('아메리카노', 236),
 ('한식', 183),
 ('커피', 161),
 ('2022고메페스타', 147),
 ('카페라떼', 133),
 ('흑돼지', 130),
 ('디저트', 113),
 ('빵', 112),
 ('갈치조림', 110),
 ('회', 104),
 ('에이드', 85),
 ('오겹살', 85),
 ('제주갈치조림', 82),
 ('전복죽', 79),
 ('흑돼지구이', 77),
 ('김치찌개', 74),
 ('고등어구이', 74),
 ('갈치구이', 70),
 ('베이커리', 68),
 ('음료', 66),
 ('바닐라라떼', 66),
 ('옥돔구이', 65),
 ('횟집', 64),
 ('해산물', 63),
 ('향토음식', 63),
 ('한치', 63),
 ('물회', 61)]

태그 데이터 대부분이 음식 메뉴에 해당  
 =>   식당에 대한 더 자세한 내용 살펴보기 위해 네이버 데이터랩 크롤링 데이터 사용

### (2) F&B 데이터랩 분위기 키워드

In [11]:
tag_list = []
dat = fb_data[-fb_data['dl_mood_list'].isnull()].reset_index()
for i in range(len(dat)):
    tags = dat['dl_mood_list'][i].split(',')
    
    for tag in tags:
        tag = keyword_processing(tag)
        tag_list.append(tag)

In [12]:
# 태그 데이터 개수 확인
word_count = Counter(tag_list)
len(word_count)

79

In [13]:
# 상위 태그 확인
word_count.most_common(30)

[('친절한', 117),
 ('깨끗한', 94),
 ('분위기좋은', 83),
 ('소박한', 74),
 ('고급진', 58),
 ('아기자기', 57),
 ('아담한', 54),
 ('아늑한', 51),
 ('이국적', 51),
 ('친절하고', 47),
 ('화려한', 45),
 ('편안한', 45),
 ('분위기있는', 39),
 ('빈티지한', 37),
 ('친절함', 33),
 ('재미있는', 31),
 ('조그마한', 24),
 ('고급스러운', 20),
 ('깔끔한인테리어', 18),
 ('따뜻한느낌', 18),
 ('모던한', 16),
 ('심플한', 15),
 ('친절하신', 15),
 ('근사한', 13),
 ('작은공간', 12),
 ('깔끔한분위기', 12),
 ('운치있는', 12),
 ('프라이빗', 11),
 ('느낌있는', 11),
 ('세련된', 10)]

### (3) F&B 데이터랩 인기토픽 키워드

In [14]:
tag_list = []
dat = fb_data[-fb_data['dl_topic_list'].isnull()].reset_index()
for i in range(len(dat)):
    tags = keyword_processing(dat['dl_topic_list'][i])
    tags = dat['dl_topic_list'][i].split(',')
    for tag in tags:
        tag = keyword_processing(tag)
        tag_list.append(tag)

In [15]:
# 태그 데이터 개수 확인
word_count = Counter(tag_list)
len(word_count)

415

In [16]:
# 상위 태그 확인
word_count.most_common(30)

[('흑돼지', 112),
 ('해변', 93),
 ('카페', 84),
 ('전복', 65),
 ('고기집', 48),
 ('갈치조림', 41),
 ('횟집', 37),
 ('김치찌개', 35),
 ('갈치', 29),
 ('시장', 28),
 ('돔베고기', 28),
 ('전복죽', 26),
 ('고등어회', 25),
 ('근고기', 25),
 ('성게미역국', 25),
 ('튀김', 25),
 ('삼겹살', 24),
 ('한라봉', 24),
 ('짬뽕', 23),
 ('피자', 23),
 ('빵집', 23),
 ('매운탕', 22),
 ('케이크', 22),
 ('고등어조림', 22),
 ('국수집', 21),
 ('해물라면', 20),
 ('냉면', 19),
 ('비빔국수', 19),
 ('디저트', 18),
 ('돈가스', 17)]

### (4)  F&B 데이터랩 찾는목적 키워드

In [17]:
tag_list = []
dat = fb_data[-fb_data['dl_why_list'].isnull()].reset_index()
for i in range(len(dat)):
    tags = keyword_processing(dat['dl_why_list'][i])
    tags = dat['dl_why_list'][i].split(',')
    for tag in tags:
        tag = keyword_processing(tag)
        tag_list.append(tag)

In [18]:
# 태그 데이터 개수 확인
word_count = Counter(tag_list)
len(word_count)

69

In [19]:
# 상위 태그 확인
word_count.most_common(30)

[('싱싱한', 224),
 ('경치', 180),
 ('드라이브', 164),
 ('신선한', 152),
 ('힐링', 148),
 ('가족여행', 130),
 ('향토음식', 90),
 ('재방문', 73),
 ('핫플레이스', 69),
 ('데이트', 53),
 ('비오는날', 44),
 ('숨은맛집', 42),
 ('착한가격', 35),
 ('등산', 35),
 ('자극적인', 32),
 ('코스요리', 32),
 ('태교여행', 25),
 ('점심특선', 24),
 ('사진찍기', 24),
 ('나들이', 21),
 ('오픈키친', 18),
 ('토속음식', 18),
 ('휴식', 16),
 ('보양식', 15),
 ('유기농', 13),
 ('숨은', 12),
 ('현지인추천', 12),
 ('숨어있는', 8),
 ('촬영지', 6),
 ('새로오픈한', 5)]

## 2. F&B 데이터 중 네이버 리뷰 수로 필터링

네이버 리뷰수가 1개라도 있는 F&B 데이터만을 추천 후보로 살펴볼 예정  
사유: mz는 리뷰 없는 곳 가지 않음 - 실없선은 위해

In [20]:
# 필요없는 열 삭제
fb_data = fb_data.drop(['BASE_YEAR', 'org_list'], axis=1)

In [22]:
fb_data = fb_data[fb_data['blog_review_num_list'] != 0]

## 3. F&B 데이터 EDA

1. visit jeju 조회수 데이터 합 => MAR+APR+MAY/SEP+OCT+NOV
2. 데이터랩20대점수 수치형으로 변경
3. 데이터랩 방문객 리뷰 - 항목별 점수로 변경
4. 데이터랩 mood,topic,why 키워드 정제  (detail_text?)

#### 추천점수 산정에 사용할 최종 데이터:  
- 수치 점수 : 데이터랩 방문객 리뷰(리뷰 개수, 블로그 리뷰 개수), 데이터랩 20대 점수, 비짓제주 조회수/like,heart 개수 => 인기도 점수 산정   
- 무드 점수 -> 20대가 좋아하는 무드가 있는지 확인하고 결정 

### (1) 조회수 데이터 총합 구하기

In [23]:
fb_20 = fb_data.copy()

In [24]:
fb_20['view_spring'] = fb_20['MAR_VIEW_CO'] + fb_20['APR_VIEW_CO'] + fb_20['MAY_VIEW_CO'] 
fb_20['view_fall'] = fb_20['SEP_VIEW_CO'] + fb_20['OCT_VIEW_CO'] + fb_20['NOV_VIEW_CO']
fb_20['view_sum'] = fb_20['view_spring'] + fb_20['view_fall']

In [25]:
fb_20 = fb_20.drop(['MAR_VIEW_CO', 'APR_VIEW_CO', 'MAY_VIEW_CO','SEP_VIEW_CO', 'OCT_VIEW_CO', 'NOV_VIEW_CO'], axis=1)

In [26]:
fb_20[['view_spring', 'view_fall', 'view_sum', 'dl_20_pop_list']].corr(method="pearson")

,view_spring,view_fall,view_sum
view_spring,1.000000,0.886922,0.966841
view_fall,0.886922,1.000000,0.975477
view_sum,0.966841,0.975477,1.000000


In [27]:
fb_20 = fb_20.drop(['view_spring', 'view_fall'], axis=1)

### (2) 데이터랩 20대 점수 수치형으로 변환

In [28]:
def pop20_processing(grade):
    grade = grade.replace('%', '')
    grade = pd.to_numeric(grade)
    return grade

In [29]:
na_index = fb_20[fb_20['dl_20_pop_list'].isnull()]['dl_20_pop_list'].index
fb_20['dl_20_pop_list'][na_index] = ["0" for i in range(len(na_index))]

In [30]:
fb_20['dl_20_grade'] = list(map(lambda x: pop20_processing(x), fb_20['dl_20_pop_list']))

In [31]:
fb_20 = fb_20.drop(['dl_20_pop_list'], axis=1)

### (3) 데이터랩 방문객 리뷰 항목별 점수로 변경

In [32]:
# 방문객 리뷰 없는 데이터 삭제
fb_20 = fb_20[fb_20['visitor_review_list'] != "error_check"].reset_index(drop=True)
fb_20 = fb_20[fb_20['visitor_review_list'] != "no_result"].reset_index(drop=True)
fb_20 = fb_20[-fb_20['visitor_review_list'].isnull()].reset_index(drop=True)

In [33]:
# 딕셔너리 형태로 변경
fb_20['review'] = ""

for i in range(len(fb_20)):
    fb_20.at[i, 'review'] = ast.literal_eval(fb_20['visitor_review_list'][i])

In [34]:
# 전체 리뷰 중 고유한 key값 리스트 추출
key_list = []
for i in range(len(fb_20)):
    key_list.append(list(fb_20['review'][i].keys()))
    
key_list = list(itertools.chain(*key_list))
key_list = list(set(key_list)) # 중복제거

In [35]:
key_list

['가성비가 좋아요',
 '품질이 좋아요',
 '음료가 맛있어요',
 '매장이 청결해요',
 '가격이 합리적이에요',
 '사진이 잘 나와요',
 '매장이 넓어요',
 '단체모임 하기 좋아요',
 '혼술하기 좋아요',
 '특별한 날 가기 좋아요',
 '반찬이 잘 나와요',
 '빵이 맛있어요',
 '특색 있는 제품이 많아요',
 '메뉴 구성이 알차요',
 '친절해요',
 '화장실이 깨끗해요',
 '집중하기 좋아요',
 '음식이 맛있어요',
 '음악이 좋아요',
 '차가 맛있어요',
 '뷰가 좋아요',
 '종류가 다양해요',
 '기본 안주가 좋아요',
 '좌석이 편해요',
 '선물하기 좋아요',
 '오래 머무르기 좋아요',
 '커피가 맛있어요',
 '컨셉이 독특해요',
 '양이 많아요',
 '주차하기 편해요',
 '혼밥하기 좋아요',
 '인테리어가 멋져요',
 '특별한 메뉴가 있어요',
 '대화하기 좋아요',
 '재료가 신선해요',
 '읽을만한 책이 많아요',
 '술이 다양해요',
 '디저트가 맛있어요']

In [36]:
# key 값을 데이터 변수로 변환
for key in key_list:
    fb_20[key] = 0

In [37]:
for idx in range(0,len(fb_20)):
    nrl = fb_20.iloc[idx]['review']
    for key, value in nrl.items():
        fb_20.at[idx,key] = value

In [38]:
#fb_20.to_csv('F&B데이터_FINAL.csv', encoding='utf-8-sig', index=False)

### (4) 20대 점수 상위 F&B 키워드 특징

- 리뷰 데이터 내 키워드 추출
- 데이터랩 무드/목적/토픽 키워드 + 리뷰 데이터 풀 만들기
- 전체 상위 키워드와 20대 상위 키워드 비교

#### (4) -1 데이터랩 무드/목적 키워드 + 리뷰 데이터 풀 만들기

- 토픽과 태그 데이터는 보통 메뉴니까 빼고 진행
- 방문자 리뷰는 각 항목별 분포를 보고 평균 이상의 점수를 가진 아이들만 해당 항목을 키워드로 넣어주기



In [39]:
fb_20['review'][1]

{'음식이 맛있어요': 102, '주차하기 편해요': 48, '친절해요': 39, '양이 많아요': 25, '재료가 신선해요': 24}

In [40]:
for i in range(len(key_list)):
    ratio = len(fb_20[fb_20[key_list[i]] != 0])/len(fb_20)
    if ratio >= 0.5 :
        print(key_list[i], end = " ")
        print(ratio)

친절해요 0.8665919282511211
음식이 맛있어요 0.757847533632287
재료가 신선해요 0.6132286995515696


In [41]:
# 카페/디저트 => 남겨두기
for i in range(len(key_list)):
    ratio = len(fb_20[fb_20[key_list[i]] != 0])/len(fb_20[fb_20['category'] == "카페/디저트"])
    if ratio >= 0.5 :
        print(key_list[i], end = " ")
        print(ratio)

가성비가 좋아요 1.7650273224043715
음료가 맛있어요 0.5846994535519126
매장이 청결해요 0.994535519125683
매장이 넓어요 0.7431693989071039
친절해요 4.224043715846994
음식이 맛있어요 3.6939890710382515
뷰가 좋아요 1.0655737704918034
커피가 맛있어요 0.8579234972677595
양이 많아요 1.7650273224043715
주차하기 편해요 0.8797814207650273
인테리어가 멋져요 0.8852459016393442
특별한 메뉴가 있어요 1.8743169398907105
재료가 신선해요 2.989071038251366
디저트가 맛있어요 0.6229508196721312


In [42]:
del_list = ['재료가 신선해요', '음식이 맛있어요', '친절해요']
grade_list = [i for i in key_list if i not in del_list]

In [43]:
grade_mean = fb_20[grade_list].mean().to_list()

In [44]:
# 각 항목별 점수가 평균 이상인 데이터에만 키워드 추가
# grade_list에 있는 애가 grade_mean 보다 큰 경우 -> 빈 열에 grade_list를 넣는거다
fb_20['visitor_review_keyword'] = ''

for i in range(len(grade_list)):
    idx = fb_20[fb_20[grade_list[i]] >= grade_mean[i]].index.to_list()
    for j in idx:
        fb_20['visitor_review_keyword'][j] = fb_20['visitor_review_keyword'][j] + ' ' + grade_list[i].replace(" ", "")

In [45]:
fb_20['keyword_all'] = [str(mood) + ' ' + str(why) + ' ' + str(visitor) for mood, why, visitor in zip(fb_20['dl_mood_list'],  fb_20['dl_why_list'], fb_20['visitor_review_keyword'])]
fb_20['keyword_all'] = fb_20['keyword_all'].apply(lambda x: x.replace(',', '').replace('0','').strip())
fb_20['keyword_all'] = fb_20['keyword_all'].apply(lambda x: x.replace('nan', '').replace('0','').strip())

In [46]:
# 키워드를 하나도 갖지 못하는 데이터 삭제
fb_20 = fb_20[fb_20['keyword_all'] != ""].reset_index(drop=True)  

In [47]:
fb_20.to_csv('F&B데이터_최종.csv', encoding='utf-8-sig', index=False)

In [48]:
# 전체 상위 키워드 추출
keyword_list = []
for i in range(len(fb_20)):
    keywords = fb_20['keyword_all'][i].split()
    for keyword in keywords:
        keyword_list.append(keyword)

In [49]:
# 키워드 개수 확인
keyword_count = Counter(keyword_list)
len(keyword_count)

174

In [50]:
# 상위 키워드 확인
keyword_count.most_common()

[('특별한메뉴가있어요', 193),
 ('싱싱한', 179),
 ('가성비가좋아요', 174),
 ('양이많아요', 152),
 ('매장이청결해요', 139),
 ('뷰가좋아요', 131),
 ('경치', 129),
 ('주차하기편해요', 127),
 ('드라이브', 126),
 ('매장이넓어요', 118),
 ('신선한', 116),
 ('커피가맛있어요', 116),
 ('인테리어가멋져요', 115),
 ('힐링', 110),
 ('가족여행', 108),
 ('음료가맛있어요', 94),
 ('친절한', 92),
 ('디저트가맛있어요', 89),
 ('향토음식', 76),
 ('깨끗한', 75),
 ('분위기좋은', 64),
 ('혼밥하기좋아요', 62),
 ('재방문', 62),
 ('소박한', 55),
 ('사진이잘나와요', 49),
 ('고급진', 48),
 ('아담한', 43),
 ('이국적', 43),
 ('핫플레이스', 43),
 ('빵이맛있어요', 42),
 ('친절하고', 42),
 ('아기자기', 40),
 ('화려한', 38),
 ('아늑한', 37),
 ('데이트', 35),
 ('비오는날', 35),
 ('숨은맛집', 34),
 ('편안한', 34),
 ('친절함', 31),
 ('등산', 29),
 ('분위기있는', 26),
 ('착한가격', 26),
 ('자극적인', 26),
 ('코스요리', 26),
 ('빈티지한', 25),
 ('재미있는', 24),
 ('조그마한', 19),
 ('나들이', 17),
 ('점심특선', 17),
 ('태교여행', 16),
 ('고급스러운', 15),
 ('토속음식', 14),
 ('모던한', 14),
 ('단체모임하기좋아요', 14),
 ('깔끔한인테리어', 13),
 ('심플한', 13),
 ('사진찍기', 13),
 ('오픈키친', 12),
 ('친절하신', 12),
 ('보양식', 12),
 ('특별한날가기좋아요', 11),
 ('근사한', 11),
 ('현지인추천', 11),
 ('프라이빗

#### (4)-3 20대점수 상위10% 키워드로 20대인기 벡터 생성

In [51]:
# 20대 점수 분포
fb_20['dl_20_grade'].describe(percentiles = [.1, .25, .5, .75, .9]) 

count    755.000000
mean      52.653572
std       37.462702
min        0.000000
10%        0.000000
25%        0.000000
50%       72.407000
75%       84.817000
90%       91.563200
max       99.834000
Name: dl_20_grade, dtype: float64

In [52]:
# 20대 점수 상위 10% 데이터 추출
top10 = fb_20[fb_20['dl_20_grade'] >= 91.563200].reset_index()
len(top10)

76

In [53]:
# 20대 상위 10% 키워드 추출
top10_keyword = []
for i in range(len(top10)):
    keywords = top10['keyword_all'][i].split()
    for keyword in keywords:
        top10_keyword.append(keyword)

In [54]:
top10_count = Counter(top10_keyword)
top10_count.most_common()

[('특별한메뉴가있어요', 26),
 ('인테리어가멋져요', 24),
 ('뷰가좋아요', 22),
 ('드라이브', 19),
 ('커피가맛있어요', 18),
 ('가성비가좋아요', 17),
 ('경치', 16),
 ('싱싱한', 16),
 ('디저트가맛있어요', 16),
 ('양이많아요', 15),
 ('신선한', 15),
 ('음료가맛있어요', 15),
 ('매장이청결해요', 13),
 ('힐링', 11),
 ('친절한', 9),
 ('주차하기편해요', 9),
 ('핫플레이스', 9),
 ('아기자기', 8),
 ('향토음식', 7),
 ('사진이잘나와요', 7),
 ('분위기좋은', 7),
 ('매장이넓어요', 7),
 ('가족여행', 6),
 ('화려한', 6),
 ('빈티지한', 6),
 ('재방문', 6),
 ('친절하고', 6),
 ('혼밥하기좋아요', 6),
 ('아늑한', 6),
 ('아담한', 5),
 ('점심특선', 5),
 ('고급진', 5),
 ('분위기있는', 5),
 ('현지인추천', 5),
 ('데이트', 4),
 ('숨은맛집', 4),
 ('빵이맛있어요', 4),
 ('오픈키친', 4),
 ('재미있는', 3),
 ('깨끗한', 3),
 ('편안한', 3),
 ('사진찍기', 3),
 ('코스요리', 3),
 ('소박한', 3),
 ('나들이', 3),
 ('느낌있는', 3),
 ('이국적', 3),
 ('비오는날', 3),
 ('깔끔한인테리어', 2),
 ('친절하신', 2),
 ('착한가격', 2),
 ('자극적인', 2),
 ('태교여행', 2),
 ('멋스러운', 2),
 ('술이다양해요', 2),
 ('심플한', 2),
 ('고급지게', 2),
 ('연예인단골', 1),
 ('감각적인', 1),
 ('세련된', 1),
 ('소박하지만', 1),
 ('토속음식', 1),
 ('보양식', 1),
 ('깔끔한분위기', 1),
 ('단체모임하기좋아요', 1),
 ('차가맛있어요', 1),
 ('서비스좋은', 1),
 ('분위기짱'

20대점수 상위10% F&B 데이터가 가진 키워드로 TF-IDF 벡터 만들기
-> 이 벡터와 유사도가 높은 F&B 여행 루트 후보로 선정

In [55]:
# top10% F&B가 가진 키워드로 TF-IDF 벡터 생성
from sklearn.feature_extraction.text import TfidfVectorizer

# top10 키워드의 문서 만들기
top10doc = " ".join(top10_keyword)

# TF-IDF 벡터 만들기
tfidf_vec = TfidfVectorizer()
top10_vec = tfidf_vec.fit_transform([top10doc])

In [56]:
#top10_vec.to_array()
top10_vec.shape

(1, 94)

In [57]:
# top10% F&B가 가진 키워드로 doc2vec 벡터 생성
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

#top10_vec.toarray()
#model = Doc2Vec(size=100,min_count=2,workers=num_cores,iter=10)

#### (4)-4  각 FB별 키워드로 벡터 생성하여 20대인기 벡터와 유사도 계산

In [58]:
place_vec = tfidf_vec.transform(fb_20['keyword_all'])
place_vec.shape

(755, 94)

In [59]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(top10_vec, place_vec).shape

(1, 755)

In [60]:
# 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity

fb_20['similarity'] = ""

for i in range(len(fb_20)):
    fb_20.at[i, 'similarity'] = cosine_similarity(top10_vec, place_vec)[0][i]

In [61]:
pd.set_option('display.max_rows', None)

In [62]:
fb_20.columns

Index(['AREA_NM_PRE', 'tags', 'like', 'heart', 'detail_text', 'LN_ADDR',
       'ROAD_ADDR', 'lon', 'lat', 'region', 'co2_max', 'name_list',
       'type_list', 'address_list', 'visitor_review_num_list',
       'blog_review_num_list', 'visitor_review_list', 'dl_mood_list',
       'dl_topic_list', 'dl_why_list', 'price', 'category', 'food_co2',
       'view_sum', 'dl_20_grade', 'review', '가성비가 좋아요', '품질이 좋아요', '음료가 맛있어요',
       '매장이 청결해요', '가격이 합리적이에요', '사진이 잘 나와요', '매장이 넓어요', '단체모임 하기 좋아요',
       '혼술하기 좋아요', '특별한 날 가기 좋아요', '반찬이 잘 나와요', '빵이 맛있어요', '특색 있는 제품이 많아요',
       '메뉴 구성이 알차요', '친절해요', '화장실이 깨끗해요', '집중하기 좋아요', '음식이 맛있어요', '음악이 좋아요',
       '차가 맛있어요', '뷰가 좋아요', '종류가 다양해요', '기본 안주가 좋아요', '좌석이 편해요', '선물하기 좋아요',
       '오래 머무르기 좋아요', '커피가 맛있어요', '컨셉이 독특해요', '양이 많아요', '주차하기 편해요', '혼밥하기 좋아요',
       '인테리어가 멋져요', '특별한 메뉴가 있어요', '대화하기 좋아요', '재료가 신선해요', '읽을만한 책이 많아요',
       '술이 다양해요', '디저트가 맛있어요', 'visitor_review_keyword', 'keyword_all',
       'similarity'],
      dtype='object')

In [63]:
fb_20[['AREA_NM_PRE', 'dl_20_grade', 'dl_mood_list',  'dl_why_list', 'similarity']].sort_values('similarity', ascending=False)

,AREA_NM_PRE,dl_20_grade,dl_mood_list,dl_why_list,similarity
55,그초록,85.018500,"아늑한, 빈티지한, 모던한, 분위기좋은","드라이브, 경치, 나들이, 힐링",0.670361
455,안녕육지사람,98.075000,"빈티지한, 분위기있는, 아기자기","드라이브, 경치, 비오는날, 힐링, 신선한",0.662612
329,블랑로쉐,31.309143,"친절한, 분위기있는, 이국적, 분위기좋은","가족여행, 드라이브, 경치, 힐링",0.644062
6,73st,78.782000,NaN,NaN,0.636318
33,고요새,47.604471,NaN,NaN,0.636318
149,도렐,0.000000,"화려한, 모던한, 이국적, 분위기좋은","핫플레이스, 경치, 오픈키친, 드라이브",0.631612
264,모래비카페,88.072000,"아늑한, 느낌있는, 아기자기, 편한의자","드라이브, 경치, 힐링, 비오는날",0.623862
238,망고홀릭제주본점,83.225000,"조그마한, 이국적, 분위기좋은","싱싱한, 신선한, 데이트, 힐링, 경치",0.619225
527,와랑와랑카페,91.788000,"아늑한, 분위기좋은, 따뜻한분위기","드라이브, 경치, 힐링, 비오는날, 유기농",0.606833
327,뵤뵤,50.956941,NaN,NaN,0.594297


In [65]:
fb_20.to_csv('data/F&B데이터_FINAL.csv', encoding='utf-8-sig', index=False)

## 3. F&B별 추천 점수 산정

F&B 평가점수 계산 (5점 만점)
1. MZ 선호도 -> 20대점수 상위10% FB mood/why 로 만든 20대인기 벡터와의 유사도
2. 제공정보량 -> 블로그 리뷰수 스케일 점수 (20대점수)
3. 탄소배출점수 -> 공간별 탄소배출 + 음식별 탄소배출량
4. 바람지도 등록여부 



#### (3)-1 탄소배출량 점수 계산 + 스케일링

In [66]:
# 탄소배출량 합하기
fb_20['co2_sum'] = fb_20['co2_max'] + fb_20['food_co2']

In [67]:
from sklearn.preprocessing import minmax_scale

# Min Max Scaling
fb_20['co2_scale'] = minmax_scale(fb_20['co2_sum'],copy=True)

#### (3)-2 친환경 점수 


In [68]:
#fb_20.to_csv('fb20.csv', encoding='utf-8-sig', index=False)

In [69]:
pla = pd.read_csv('data/PLA사용카페.csv', encoding='utf-8-sig')
baram = pd.read_csv('data/제주바람지도.csv', encoding='utf-8-sig')
bluecup = pd.read_csv('data/제주푸른컵지도.csv', encoding='utf-8-sig')
vegan = pd.read_csv('data/제주비건식당크롤링.csv', encoding='utf-8-sig')

In [70]:
def name_preprocessing(name):
    name = name.replace(' (', '(').replace(') ', ')').strip()
    name = name.replace('(주)','').replace("·",".").lower()
    name = re.sub('[\(\)㈜!@#$%^&*{}|:;\'\"/><,~`+=_ ]','', name)
    name = re.sub('[\[.\]]','',name)
    return name.strip()

In [71]:
#주소 형식 맞추기
def addr_preprocessing(addr):
    addr = re.sub('제주 서귀포시','제주특별자치도 서귀포시',addr)
    addr = re.sub('제주 제주시','제주특별자치도 제주시',addr)

    while (addr.split()[-1][-1].isdigit() == False) : #상세주소 제거
        if len(addr.split()) <= 3 : break
            
        addr = " ".join(addr.split()[0:-1]) #하나빼기

    addr = "".join(addr.split()) #공백다 없애기
    return addr.strip() #앞뒤정리

##### (1) PLA 사용카페

In [72]:
eco_list = []

#이름이 일치하는 경우
name_list = list(map(lambda x : name_preprocessing(x), pla['이름']))
set(eco_list).union(set(fb_20[fb_20['AREA_NM_PRE'].isin(name_list)].index))

#포함되는 경우
for i in range(0, len(name_list)) :
    idx = fb_20[fb_20['AREA_NM_PRE'].str.contains(name_list[i])].index
    if len(idx) >0 :
        eco_list.append(idx.values[0])

for i in range(0, len(fb_20)) :
    if len(pla[pla['이름'].str.contains(fb_20.AREA_NM_PRE[i])]) >0 : 
        eco_list.append(i)

In [73]:
#주소가 일치하는 경우
pla['주소'] =  "제주특별자치도 " + pla['주소']
addr_list = pla['주소']
set(eco_list).union(set(fb_20[fb_20['ROAD_ADDR'].isin(addr_list)].index))

#포함되는 경우
for i in range(0, len(addr_list)) :
    idx = fb_20[fb_20['ROAD_ADDR'].str.contains(addr_list[i])].index
    if len(idx) >0 :
        eco_list.append(idx.values[0])

for i in range(0, len(fb_20)) :
    if len(pla[pla['주소'].str.contains(fb_20.ROAD_ADDR[i])]) >0 : 
        eco_list.append(i)

##### (2) 제주 바람지도


In [74]:
baram.columns = ['area_name', 'tel', 'instagram', 'detail', 'category']

In [75]:
#이름이 일치하는 경우
name_list = list(map(lambda x : name_preprocessing(x), baram['area_name']))
set(eco_list).union(set(fb_20[fb_20['AREA_NM_PRE'].isin(name_list)].index))

#포함되는 경우
for i in range(0, len(name_list)) :
    idx = fb_20[fb_20['AREA_NM_PRE'].str.contains(name_list[i])].index
    if len(idx) >0 :
        eco_list.append(idx.values[0])

for i in range(0, len(fb_20)) :
    if len(baram[baram['area_name'].str.contains(fb_20.AREA_NM_PRE[i])]) >0 : 
        eco_list.append(i)

##### (3) 제주 푸른컵지도

In [76]:
#이름이 일치하는 경우
name_list = list(map(lambda x : name_preprocessing(x), bluecup['이름']))
set(eco_list).union(set(fb_20[fb_20['AREA_NM_PRE'].isin(name_list)].index))

#포함되는 경우
for i in range(0, len(name_list)) :
    idx = fb_20[fb_20['AREA_NM_PRE'].str.contains(name_list[i])].index
    if len(idx) >0 :
        eco_list.append(idx.values[0])

for i in range(0, len(fb_20)) :
    if len(bluecup[bluecup['이름'].str.contains(fb_20.AREA_NM_PRE[i])]) >0 : 
        eco_list.append(i)

##### (4) 제주 비건식당 

In [77]:
#이름이 일치하는 경우
name_list = list(map(lambda x : name_preprocessing(x), vegan['title']))
set(eco_list).union(set(fb_20[fb_20['AREA_NM_PRE'].isin(name_list)].index))

#포함되는 경우
for i in range(0, len(name_list)) :
    idx = fb_20[fb_20['AREA_NM_PRE'].str.contains(name_list[i])].index
    if len(idx) >0 :
        eco_list.append(idx.values[0])

for i in range(0, len(fb_20)) :
    if len(vegan[vegan['title'].str.contains(fb_20.AREA_NM_PRE[i])]) >0 : 
        eco_list.append(i)

In [78]:
#중복제거 및 처리
eco_list = set(eco_list)
fb_20.iloc[list(eco_list)]

,AREA_NM_PRE,tags,like,heart,detail_text,LN_ADDR,ROAD_ADDR,lon,lat,region,...,대화하기 좋아요,재료가 신선해요,읽을만한 책이 많아요,술이 다양해요,디저트가 맛있어요,visitor_review_keyword,keyword_all,similarity,co2_sum,co2_scale
513,오캄,카페 베이커리 유기농빵 음식 빵 바게트 수제마카롱 아메리카노 와플 음식 빵 바게트 ...,0,1,민오름 뒤에 위치한 오캄은 스트레스를 받지 않고 심신이 편안한 상태를 추구하는 베이...,제주특별자치도 제주시 오라이동 729-41,제주특별자치도 제주시 정실3길 118,126.509351,33.471515,오라이동,...,0,0,0,0,0,빵이맛있어요,빵이맛있어요,0.053692,1.213742,0.036717
135,덕성원,꽃게짬뽕 꽃게볶음 탕수육 중식 음식 볶음밥 2022고메페스타,2,51,제주도를 대표하는 '도민 중국집'이라 할 수 있는 곳이다. 중문과 제주시 등 도 전...,제주특별자치도 서귀포시 서귀동 474,제주특별자치도 서귀포시 태평로401번길 4,126.563388,33.244910,서귀동,...,0,0,0,0,0,매장이넓어요 양이많아요 특별한메뉴가있어요,고급진 가족여행 싱싱한 코스요리 숨어있는 매장이넓어요 양이많아요 특별한메뉴가있어요,0.39573,1.790119,0.060857
528,와르다레스토랑,할랄 무슬림프렌들리 음식점 예맨 음식 케밥 샐러드 식당,0,2,제주 원도심을 여행한다면 예멘 난민 출신 셰프가 직접 요리하는 와르다(Wardah)...,제주특별자치도 제주시 삼도이동 148-3,제주특별자치도 제주시 관덕로8길 24-1,126.523113,33.511039,삼도이동,...,0,26,0,0,0,혼밥하기좋아요 특별한메뉴가있어요,혼밥하기좋아요 특별한메뉴가있어요,0.303731,1.573033,0.051765
658,타코마씸,멕시코요리 타코 흑돼지타코 멕시칸 음식 에이드 핫도그 감자튀김 칵테일 멕시칸 음식 ...,0,9,"월정리 해변 근처에 있는 멕시코음식점이다. 2인석 테이블이 2개, 4인석 테이블이 ...",제주특별자치도 제주시 구좌읍 월정리 13,제주특별자치도 제주시 구좌읍 해맞이해안로 474,126.797208,33.555080,구좌읍,...,0,24,0,0,0,,느낌있는 분위기있는 이국적 향토음식 핫플레이스 경치 드라이브,0.314555,1.839272,0.062915
414,소소희,영어마을 샤워도우 치아바타 대정읍 베어커리 빵지순례 음식 카페 빵 베이커리,0,0,오랫동안 배양한 발효종을 사용하여 만든 빵 대정읍 영어도시에 위치한 베이커리 전문...,제주특별자치도 서귀포시 대정읍 보성리 175-6,제주특별자치도 서귀포시 대정읍 영어도시로 27-2,126.278154,33.274898,대정읍,...,0,0,0,0,0,빵이맛있어요,빵이맛있어요,0.053692,1.213742,0.036717
672,톰톰카레,카레 야채카레 콩카레 반반카레 음식,1,52,"제주도는 화려하다기 보다 수수한 매력이 있다. 톰톰카레의 메뉴는 야채카레, 콩카레,...",제주특별자치도 제주시 구좌읍 평대리 2033-1,제주특별자치도 제주시 구좌읍 해맞이해안로 1112,126.837300,33.537317,구좌읍,...,0,100,0,0,0,혼밥하기좋아요 특별한메뉴가있어요,아늑한 따뜻한느낌 아담한 소박한 핫플레이스 신선한 드라이브 혼밥하기좋아요 특별한메...,0.402694,1.839272,0.062915
34,공백,휴식치유 카페 동복 구좌 음식 아메리카노 레몬에이드 에이드 에스프레소 바닐라라떼 카...,1,5,"함덕과 김녕 사이 조용한 마을 동복리에 위치한 ""공백""은 자연 그대로의 제주를 느낌...",제주특별자치도 제주시 구좌읍 동복리 1568-1,제주특별자치도 제주시 구좌읍 동복로 83,126.715094,33.553392,구좌읍,...,0,0,0,0,119,사진이잘나와요 뷰가좋아요 커피가맛있어요 인테리어가멋져요 디저트가맛있어요,사진이잘나와요 뷰가좋아요 커피가맛있어요 인테리어가멋져요 디저트가맛있어요,0.522261,0.424144,0.003647
293,미쿠니,커피 밀크티 디저트 제주카페 음식 식당 카페 제주카페 음식 식당 카페,0,1,미쿠니의 낮과 미쿠니의 황혼으로 운영되는 공간 삼양해수욕장 인근에 위치한 미쿠니는...,제주특별자치도 제주시 삼양일동 1779,제주특별자치도 제주시 서흘길 41,126.589629,33.529083,삼양일동,...,0,0,0,0,241,음료가맛있어요 뷰가좋아요 커피가맛있어요 인테리어가멋져요 디저트가맛있어요,음료가맛있어요 뷰가좋아요 커피가맛있어요 인테리어가멋져요 디저트가맛있어요,0.570285,0.488240,0.006332
552,우무,푸딩 우뭇가사리 수제푸딩 음식 밀크티 커스터드푸딩 음식 밀크티 커스터드푸딩,0,34,제주의 해녀가 채취한 우뭇가사리를 오랫동안 끓여 만든 수제 푸딩을 판매한다. 입속에...,제주특별자치도 제주시 한림읍 옹포리 324-3,제주특별자치도 제주시 한림읍 한림로 542-1,126.256444,33.405900,한림읍,...,0,0,0,0,913,매장이청결해요 인테리어가멋져요 특별한메뉴가있어요 디저트가맛있어요,매장이청결해요 인테리어가멋져요 특별한메뉴가있어요 디저트가맛있어요,0.530213,0.411831,0.003132
297,바그다드,무슬림 인도요리 난 음식 카레 치킨,0,4,제주시청 인근에 위치한 인도 요리 전문점으로 현지스타일의 맛을 그대로 구현한다. ...,제주특별자치도 제주시 이도이동 1188-16,제주특별자치도 제주시 서광로32길 38,126.529636,33.497486,이도이동,...,0,32,0,0,0,인테리어가멋져요 특별한메뉴가있어요,아늑한 빈티지한 이국적 친절한 나들이 재방문 데이트 신선한 인테리어가멋져요 특별한...,0.432966,1.622549,0.053839


##### (5) 친환경점수 (0,1) 삽입

In [79]:
fb_20['eco_YN'] = 0
fb_20['eco_YN'].iloc[list(eco_list)] = 1

In [80]:
fb_20[fb_20['eco_YN'] == 1]['AREA_NM_PRE']

34             공백
50            그루브
55            그초록
56        글라글라하와이
135           덕성원
207    리치망고도두해안로점
293           미쿠니
297          바그다드
309          뱅인타코
330          블랑제리
331        블루메베이글
334         비어라운드
341            빵귿
343           쁠랑뜨
414           소소희
451       아이갓에브리씽
513            오캄
528       와르다레스토랑
552            우무
588          인그리드
590         인디언키친
593       인카페온더비치
615           제레미
623         제주돌창고
624       제주돔베막국수
658          타코마씸
672          톰톰카레
691          하하호호
Name: AREA_NM_PRE, dtype: object

#### (3)-3 블로그 리뷰수 스케일링

In [81]:
fb_20['blog_review_num_scale'] = minmax_scale(fb_20['blog_review_num_list'], copy=True)

#### (3)-4 최종 F&B 점수 매기기


In [82]:
fb_20[['AREA_NM_PRE', 'co2_scale', 'similarity', 'blog_review_num_scale', 'eco_YN']].head()

,AREA_NM_PRE,co2_scale,similarity,blog_review_num_scale,eco_YN
0,2093하우스,0.056804,0.290834,0.044319,0
1,24시누름돌김치찌개,0.085393,0.218306,0.003850,0
2,24시뼈다귀탕동홍점,0.327342,0.093962,0.001690,0
3,24시얼큰시원생고기김치찌개노형점,0.089743,0.228193,0.001408,0
4,24시전주명가콩나물국밥,0.336821,0.216996,0.007230,0


In [83]:
fb_20['rec_score'] = (1-fb_20['co2_scale']) + fb_20['similarity'] + fb_20['blog_review_num_scale'] + fb_20['eco_YN']
fb_20['rec_score'] = pd.to_numeric(fb_20['rec_score'])

In [84]:
fb_20['rec_score'].describe()

count    755.000000
mean       1.360800
std        0.295386
min        0.519783
25%        1.201532
50%        1.339394
75%        1.481124
max        2.893279
Name: rec_score, dtype: float64

In [85]:
fb_20[['AREA_NM_PRE',  'blog_review_num_scale','similarity', 'co2_scale', 'eco_YN', 'rec_score', 'category']].sort_values('rec_score', ascending=False)

,AREA_NM_PRE,blog_review_num_scale,similarity,co2_scale,eco_YN,rec_score,category
552,우무,0.366197,0.530213,0.003132,1,2.893279,카페/디저트
55,그초록,0.065258,0.670361,0.006332,1,2.729288,카페/디저트
691,하하호호,0.207136,0.570658,0.099180,1,2.678615,패스트푸드
34,공백,0.153052,0.522261,0.003647,1,2.671666,카페/디저트
293,미쿠니,0.040376,0.570285,0.006332,1,2.604329,카페/디저트
593,인카페온더비치,0.045070,0.560309,0.006332,1,2.599048,카페/디저트
50,그루브,0.044883,0.558279,0.006332,1,2.596830,카페/디저트
588,인그리드,0.077840,0.476521,0.000984,1,2.553378,카페/디저트
615,제레미,0.043380,0.469809,0.006332,1,2.506858,카페/디저트
623,제주돌창고,0.017934,0.489944,0.006332,1,2.501546,카페/디저트


In [86]:
fb_20.to_csv('data/F&B추천점수.csv', encoding='utf-8-sig', index=False)